# Distributions

In [13]:
import pandas as pd

data = pd.read_csv('Dataframe_UDs.csv', sep=',', skiprows=4)

In [18]:
data

,Up_dur,Down_dur,id,level
count,1553.000000,1553.000000,1553.000000,1553
unique,NaN,NaN,NaN,3
top,NaN,NaN,NaN,light
freq,NaN,NaN,NaN,534
mean,0.539924,0.679102,5.835802,NaN
std,0.289845,0.644969,2.386486,NaN
min,0.083232,0.078336,0.000000,NaN
25%,0.322967,0.267833,3.000000,NaN
50%,0.479400,0.385156,6.000000,NaN
75%,0.698700,0.831300,8.000000,NaN


## Discrete

## Continuous

## Histograms

## KDE

## Outliers

## Stats

### Mean, median, mode

### standard deviation

### range, quantiles

## (t-test)